In [501]:
import cv2 as cv
import numpy as np
from math import exp

In [535]:
hough_image = cv.imread('original_imgs/hough.jpg',0)
v_org_image = cv.imread('original_imgs/hough.jpg')
d_org_image = cv.imread('original_imgs/hough.jpg')
circle_org_image = cv.imread('original_imgs/hough.jpg')

sobel_x = [[-1, 0, 1], [-2,0,2], [-1,0,1]]
sobel_y = [[-1, -2, -1], [0,0,0], [1,2,1]]


In [503]:
def gaussian(x, mu, sigma):
  return exp( -(((x-mu)/(sigma))**2)/2.0 )

def createKernel(sigma):
    
    
  #kernel_height, kernel_width = 7, 7
    kernel_radius = 3 # for an 7x7 filter
  

  # compute the actual kernel elements
    hkernel = [gaussian(x, kernel_radius, sigma) for x in range(2*kernel_radius+1)]
    vkernel = [x for x in hkernel]
    kernel2d = [[xh*xv for xh in hkernel] for xv in vkernel]

  # normalize the kernel elements
    kernelsum = sum([sum(row) for row in kernel2d])
    kernel2d = [[x/kernelsum for x in row] for row in kernel2d]
    return kernel2d


def createImage(img):
    kernel2d = np.array([[1/16.,1/8.,1/16.],[1/8.,1/4.,1/8.],[1/16.,1/8.,1/16.]])
    height,width=img.shape
    img2dhororg=np.zeros(img.shape)
    pixel_x=0
    for x in range(3,height-3):
        for y in range(3,width-3):
            for i in range(0,3):
                for j in range(0,3):
                    pixel_x=pixel_x+kernel2d[i][j]*img[x-3+i][y-3+j]
                                    
        
            img2dhororg[x][y]=pixel_x
            pixel_x=0
    return img2dhororg

In [504]:
def getSobelImage(img,sobel):
    height, width= img.shape
    sobelImage=[[0 for col in range(width)] for row in range(height)]
    for x in range(1,height-1):
        for y in range(1,width-1):
            pixel_x =           (sobel[0][0] * img[x-1][y-1]) + \
                            (sobel[0][1] * img[x-1][y]) + \
                            (sobel[0][2] * img[x-1][y+1]) + \
                            (sobel[1][0] * img[x][y-1])   +\
                             (sobel[1][1] * img[x][y])   + \
                             (sobel[1][2] * img[x][y+1]) + \
                             (sobel[2][0] * img[x+1][y-1]) + \
                             (sobel[2][1] * img[x+1][y]) + \
                             (sobel[2][2] * img[x+1][y+1])
            sobelImage[x][y]=pixel_x
    return np.asarray(sobelImage)        
      
def normalizeMatrix(img):
    h,w=img.shape
    currMax=0
    for x in range(0,h):
        for y in range(0,w):
            if (img[x][y]<0):
                img[x][y]= 0-img[x][y]
            if (currMax<img[x][y]):
                currMax=img[x][y]

    for i in range(0,h):
        for j in range(0,w):
            img[i][j]=(img[i][j]/currMax)*255

    return img


In [505]:
def generateHoughLines(image):
    theta = np.deg2rad(np.arange(-90.0, 90.0))
    #theta = np.deg2rad([-90,-60,-45,-30,-15,0,15,30,45,60,90])
    print(image.shape)
    width,height = image.shape
    len_diagonal = int(np.ceil(np.sqrt(width * width + height * height)))
    rhos = np.linspace(-len_diagonal, len_diagonal, len_diagonal * 2.0)

    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    len_thetas = len(theta)

    print(len_diagonal)
    accumulator = np.zeros((2 * len_diagonal, len_thetas), dtype=np.uint64)
    y_idx, x_idx = np.nonzero(image)
    print(len_thetas)

    for i in range(len(x_idx)):
        x = x_idx[i]
        y = y_idx[i]

        for t_idx in range(len_thetas):
            rho = int(round(x * cos_theta[t_idx] + y * sin_theta[t_idx]) + len_diagonal)
            #print(rho)
            accumulator[rho, t_idx] += 1

    return accumulator, theta, rhos

In [506]:
def thresholdSobel(Image):
    img = np.zeros(Image.shape)
    h1,w1 = Image.shape
    for i in range(0,h1):
        for j in range(0,w1):
            if(Image[i][j]>40):
                img[i][j]=255
    return img            

In [507]:
#kernel = createKernel(1/1.414)
hough_gaussian = createImage(hough_image)

In [508]:

sobelHoughX=getSobelImage(hough_image,sobel_x)
sobelHoughY=getSobelImage(hough_image,sobel_y)
hough_edge_x=normalizeMatrix(sobelHoughX)
hough_edge_y=normalizeMatrix(sobelHoughY)

threshold_sobel_x = thresholdSobel(hough_edge_x)


In [509]:
cv.imwrite("threshold_sobel_g"+'.jpg',threshold_sobel_x)

True

In [510]:
accumulator, thetas, rhos = generateHoughLines(threshold_sobel_x)

(477, 666)
820
180


/anaconda3/envs/ML_Tensor_Keras/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  import sys


In [511]:
print(accumulator.shape)
#print(accumulator[860,179])

print(np.argmax(accumulator))
idx = np.argmax(accumulator)
r = int(round(idx / accumulator.shape[1]))
rho = rhos[r]
print(rhos)

theta = thetas[idx % accumulator.shape[1]]
print(theta)
print(rho, np.rad2deg(theta))
a=accumulator
nd = np.unravel_index(np.argmax(a, axis=None), a.shape)
print(nd)

#print(accumulator[1244][135])



(1640, 180)
147870
[-820.         -818.99938987 -817.99877974 ...  817.99877974  818.99938987
  820.        ]
0.0
2.501525320317228 0.0
(821, 90)


In [512]:
def hough_peaks1(H, num_peaks):
    indices =  np.argpartition(H.flatten(), -2)[-num_peaks:]
    return np.vstack(np.unravel_index(indices, H.shape)).T

In [536]:
peaks = hough_peaks1(accumulator,200)
v_peaks = peaks[peaks[:,1]>87]
v_peaks = v_peaks[v_peaks[:,1]<110]
#v_peaks = v_peaks[v_peaks[:,0]>910]
v_peaks = v_peaks[:40,:]
d_peaks = peaks[peaks[:,1]<56]
d_peaks = d_peaks[d_peaks[:,0]>714]
d_peaks = d_peaks[:20,:]
#print(d_peaks)

#print(v_peaks)
#theta = thetas[peaks[2][1]]
#theta

hough_lines_draw(v_org_image, v_peaks,rhos, thetas)
hough_lines_draw(d_org_image, d_peaks,rhos, thetas)

In [514]:
def hough_lines_draw(img, indicies, rhos, thetas):
    for i in range(len(indicies)):
        
        rho = rhos[indicies[i][0]]
        theta = thetas[indicies[i][1]]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

In [537]:
cv.imwrite("red_line"+'.jpg',v_org_image)
cv.imwrite("blue_lines"+'.jpg',d_org_image)

True

In [516]:
cv.imwrite("accumulator"+'.jpg',accumulator)

True

# Hough Circle

In [517]:
def thresholdSobelCircle(Image):
    img = np.zeros(Image.shape)
    h1,w1 = Image.shape
    for i in range(0,h1):
        for j in range(0,w1):
            if(Image[i][j]>55):
                img[i][j]=255
    return img 

In [518]:
threshold_sobel_y = thresholdSobelCircle(hough_edge_y)
#cv.imwrite("threshold_sobel_y"+'.jpg',threshold_sobel_y)

In [519]:
def generateHoughCircles(image):
    radius = 20
    theta = np.deg2rad(np.arange(0.0, 360.0))
    #theta = np.deg2rad([-90,-60,-45,-30,-15,0,15,30,45,60,90])
    print(image.shape)
    width,height = image.shape
    len_diagonal = int(np.ceil(np.sqrt(width * width + height * height)))
    #rhos = np.linspace(-len_diagonal, len_diagonal, len_diagonal * 2.0)

    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    len_thetas = len(theta)

    print(len_diagonal)
    accumulator_circle = np.zeros((len_diagonal, len_diagonal), dtype=np.uint64)
    y_idx, x_idx = np.nonzero(image)
    print(len_thetas)

    for i in range(len(x_idx)):
        x = x_idx[i]
        y = y_idx[i]

        for t_idx in range(len_thetas):
            a = int(round(x - radius * cos_theta[t_idx]) )
            b = int(round(y - radius * sin_theta[t_idx]) )
            #rho = int(round(x * cos_theta[t_idx] + y * sin_theta[t_idx]) + len_diagonal)
            #print(rho)
            accumulator_circle[a, b] += 1

    return accumulator_circle, theta

In [520]:
accumulator_circle, thetas_circle = generateHoughCircles(threshold_sobel_y )

(477, 666)
820
360


In [521]:
cv.imwrite("accumulator_circle"+'.jpg',accumulator_circle)

True

In [522]:
def hough_peaks_circle(H, num_peaks):
    indices =  np.argpartition(H.flatten(), -2)[-num_peaks:]
    return np.vstack(np.unravel_index(indices, H.shape)).T

In [523]:
peaks_circle = hough_peaks_circle(accumulator_circle,150)
#print(peaks_circle)

In [524]:

def displayCircles(peaks):
    for i in range(len(peaks)):
        
        cx = peaks[i][0]
        cy = peaks[i][1]
        #print(cx,cy)
        cv.circle(circle_org_image, (cx, cy), 20, (0, 255, 0), -1)

In [525]:
displayCircles(peaks_circle)

In [526]:
cv.imwrite("coin"+'.jpg',circle_org_image)

True